# Домашнее задание №1: линейная регрессия и векторное дифференцирование (10 баллов).

* Максимальное количество баллов за задания в ноутбуке - 11, но больше 10 оценка не ставится, поэтому для получения максимальной оценки можно сделать не все задания.

* Некоторые задания будут по вариантам (всего 4 варианта). Чтобы выяснить свой вариант, посчитайте количество букв в своей фамилии, возьмете остаток от деления на 4 и прибавьте 1.

In [1]:
import numpy as np

## Многомерная линейная регрессия из sklearn

Применим многомерную регрессию из sklearn для стандартного датасета

In [2]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples = 10000)
print(X.shape, y.shape)

(10000, 100) (10000,)


У нас 10000 объектов и 100 признаков. Для начала решим задачу аналитически "из коробки".

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = LinearRegression().fit(X, y)
print(mean_squared_error(y, reg.predict(X)))

2.0150149549608656e-25


Теперь попробуем обучить линейную регрессию методом градиентного спуска "из коробки"

In [4]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor(alpha=1e-15, tol=1e-25, max_iter=10000).fit(X, y)
print(mean_squared_error(y, reg.predict(X)))

1.5067096265590522e-25


***Задание 1 (0.5 балла).*** Объясните, чем вызвано различие двух полученных значений метрики?

***Задание 2 (0.5 балла).*** Подберите гиперпараметры в методе градиентного спуска так, чтобы значение MSE было близко к значению MSE, полученному при обучении LinearRegression.

LinearRegression использует поиск коэфициентов аналитическим способом, что даёт максимальную точность, но требует продолжительных вычислений


## Ваша многомерная линейная регрессия

***Задание 3 (5 баллов)***. Напишите собственную многомерную линейную регрессию, оптимизирующую MSE методом *градиентного спуска*. Для этого используйте шаблонный класс. 

Критерий останова: либо норма разности весов на текущей и предыдущей итерациях меньше определенного значения (первый и третий варианты), либо модуль разности функционалов качества (MSE) на текущей и предыдущей итерациях меньше определенного значения (второй и четвертый варианты). Также предлагается завершать обучение в любом случае, если было произведено слишком много итераций.

***Задание 4 (2 балла)***. Добавьте l1 (первый и второй варианты) или l2 (третий и четвертый варианты) регуляризацию. 

In [14]:
class LinearRegression(object):
    def __init__(self, alpha=1e-3, l1_ratio=0, l2_ratio=0, tol=0.001, max_iter=1000, stop_by='weights'):
        '''
        alpha — шаг обучения
        l1_ratio — коэффициент регуляризации L1
        l2_ratio — коэффициент регуляризации L2
        tol — порог останова
        max_iter — максимальное число итераций
        stop_by — критерий останова: 'weights' или 'loss'
        '''
        self.alpha = alpha
        self.l1_ratio = l1_ratio
        self.l2_ratio = l2_ratio
        self.tol = tol
        self.max_iter = max_iter
        self.stop_by = stop_by  # 'weights' or 'loss'
        self.weights = None
        self.bias = None

    def _mse(self, X, y):
        '''
        Вычисление среднеквадратичной ошибки
        '''
        predictions = X @ self.weights + self.bias
        return np.mean((predictions - y) ** 2)

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0.0

        prev_loss = float('inf')
        prev_weights = np.copy(self.weights)

        for i in range(self.max_iter):
            y_pred = X @ self.weights + self.bias
            error = y_pred - y

            l1 = self.l1_ratio * np.sign(self.weights)
            l2 = 2 * self.l2_ratio * self.weights

            # Градиенты
            dw = (2 / n_samples) * (X.T @ error) + l1 + l2
            db = (2 / n_samples) * np.sum(error)

            # Обновление весов
            self.weights -= self.alpha * dw
            self.bias -= self.alpha * db

            # Критерии останова
            if self.stop_by == 'weights':
                weight_diff = np.linalg.norm(self.weights - prev_weights)
                if weight_diff < self.tol:
                    break
                prev_weights = np.copy(self.weights)
            elif self.stop_by == 'loss':
                current_loss = np.mean(error ** 2)
                if abs(prev_loss - current_loss) < self.tol:
                    break
                prev_loss = current_loss

    def predict(self, X):
        return X @ self.weights + self.bias

In [9]:
my_reg = LinearRegression(alpha=0.04, tol=1e-3, max_iter=1000)
my_reg.fit(X, y)
print(mean_squared_error(y, my_reg.predict(X)))
assert mean_squared_error(y, my_reg.predict(X)) < 1e-3
print('You are amazing! Great work!')

0.00013313831140211477
You are amazing! Great work!


***Задание 5 (1 балл)***. Обучите линейную регрессию из коробки

* с l1-регуляризацией (from sklearn.linear_model import Lasso, **первый и второй вариант**) или с l2-регуляризацией (from sklearn.linear_model import Ridge, **третий и четвертый вариант**)
* со значением параметра регуляризации **0.1 - для первого и третьего варианта, 0.01 - для второго и четвертого варианта**. 

Обучите вашу линейную регрессию с тем же значением параметра регуляризации и сравните результаты. Сделайте выводы.

In [17]:
#your code here
from sklearn.linear_model import Lasso

lasso_reg = Lasso(alpha=0.04, tol=1e-3, max_iter=1000)
lasso_reg.fit(X, y)
lasso_mse = mean_squared_error(y, lasso_reg.predict(X))

my_reg = LinearRegression(alpha=0.04, l1_ratio=0.04, tol=1e-3, max_iter=1000)
my_reg.fit(X, y)
my_mse = mean_squared_error(y, my_reg.predict(X))

print(f"MSE Lasso: {lasso_mse}")
print(f"MSE наша реализация: {my_mse}")

# Сравниваем количество ненулевых весов
print(f"\nКоличество ненулевых весов в Lasso: {np.sum(lasso_reg.coef_ != 0)}")
print(f"Количество ненулевых весов в нашей реализации: {np.sum(my_reg.weights != 0)}")


MSE Lasso: 0.01617951779635554
MSE наша реализация: 0.004119128564820983

Количество ненулевых весов в Lasso: 10
Количество ненулевых весов в нашей реализации: 100


Наше решение является решением через градиентный спуск, а Lasso использует координатный спуск с вычислением аналитического решения для каждого веса в отдельности, из-за чего качество обучения из коробки для сгенерированной выборки выше

***Задание 6* (1 балл).***
Пусть $P, Q \in \mathbb{R}^{n\times n}$. Найдите $\nabla_Q tr(PQ)$

***Задание 7* (1 балл).***
Пусть $x, y \in \mathbb{R}^{n}, M \in \mathbb{R}^{n\times n}$. Найдите $\nabla_M x^T M y$

Решения заданий 6 и 7 можно написать на листочке и отправить в anytask вместе с заполненным ноутбуком.

### Решение задания 6

Найдем $\nabla_Q tr(PQ)$:

1. След матрицы $tr(PQ)$ можно записать как:
$$tr(PQ) = \sum_{i=1}^n \sum_{j=1}^n P_{ij}Q_{ji}$$

2. Частная производная по элементу $Q_{kl}$:
$$\frac{\partial tr(PQ)}{\partial Q_{kl}} = P_{lk}$$

3. Следовательно, градиент:
$$\nabla_Q tr(PQ) = P^T$$

### Решение задания 7

Найдем $\nabla_M x^T M y$:

1. Выражение $x^T M y$ можно записать как:
$$x^T M y = \sum_{i=1}^n \sum_{j=1}^n x_i M_{ij} y_j$$

2. Частная производная по элементу $M_{kl}$:
$$\frac{\partial (x^T M y)}{\partial M_{kl}} = x_k y_l$$

3. Следовательно, градиент:
$$\nabla_M x^T M y = x y^T$$